In [ ]:
import os

import pandas as pd

private_key_path = "../private/private_key.pem"
key = Fernet.generate_key()
f = Fernet(key)

# Sauvegarder la clé
with open(private_key_path, "wb") as key_file:
    key_file.write(key)


## Chiffrement du fichier Excel contenant les données des étudiants

In [ ]:
def encrypt_student_list(key_path, excel_path):
    with open(key_path, "rb") as key_file:
        key = key_file.read()
    df = pd.read_excel(excel_path)
    columns = df.columns
    # Convertir le DataFrame en un tableau structuré (structured array) de NumPy
    structured_array = df.to_records(index=False)

    # Convertir chaque dictionnaire en une chaîne de caractères
    list_of_strings = []
    for d in structured_array:
        for elm in d:
            list_of_strings.append(str(elm))
    
    f = Fernet(key)
    cryptList = [f.encrypt(elm.encode()) for elm in list_of_strings]
    res = []
    for i in range(0, len(cryptList), 4):
        res.append([cryptList[i], cryptList[i+1], cryptList[i+2], cryptList[i+3]])
    df = pd.DataFrame(res, columns=columns)
    crypt_xlsx = "../private/list_crypt.xlsx"
    df.to_excel(crypt_xlsx, index=False)
    return crypt_xlsx

encrypt_student_list(private_key_path, "../private/login_excel.xlsx")

In [92]:
df = pd.read_excel("../private/list_crypt.xlsx")

with open(private_key_path, "rb") as key_file:
    key = key_file.read()
f = Fernet(key)
structured_array = df.to_records(index=False)
list_of_strings = []
for d in structured_array:
    for elm in d:
        list_of_strings.append(str(elm))
list_of_strings

["b'gAAAAABlQ5lVOqna1cC8IyS1uxRXheU7mLIfW3Iq-m6Q_LUWvF9v3sQPzmJVfukycFzIVSDqPNpKj5-RqIZg2BtW7ll3Rnm6Gw=='",
 "b'gAAAAABlQ5lV2HJX6k4OT6c1ewlgkhKS3Eib2rtQqsETJg40eH45bEQ5nYmR3E9mYIo5JxmaLDygJzhlxtM4OjPfEqeOgmL3jg=='",
 "b'gAAAAABlQ5lVIdTLMMhTOzv42KGWq9_j-lAa53RYFn_4EqimFAbfzK9hYhlitVyh_9mD5BoYxsibRnbJKUfG829nER3z9vnXTw=='",
 "b'gAAAAABlQ5lVDkrrmFXxocD91mfwMaZeu0_OcDYlG6HhAh62mZD_y20v6hvRGitcHovNA_6F-fIUeoCacMxdbL1-kRIhqNlxGA=='",
 "b'gAAAAABlQ5lVVxXj58yXkkSvelaQmrHTIU2MFZnCdlm2DR4as2t15oa5njf9t4OsZBa8gCEqBICuKyZDE3Vy9MMgNjktgUUFHA=='",
 "b'gAAAAABlQ5lVWFz3x8bvreRV15tGQND_5hsVlRz2LGx9WJwIWiDCdNL0cW12ANZ-uKrvfDzMfExcq2sreINuw4aiAmzcMJhIoA=='",
 "b'gAAAAABlQ5lVsXKlBEP40ZXxGVY5KGbgu_KT5MPSotOA4DSi0-4CMcMgfDrFAUnP-VBsmyOQBbswZTjCWCPuDBo-IQ5tArJGdw=='",
 "b'gAAAAABlQ5lVX4JZg1sVwLgg1BDDcKtQOdZZ599W9uQhmrFEueKf_3dztxHWBIiAhhF8or5dAo_aH_XJsaBRAgGO8ZqDglHuVg=='",
 "b'gAAAAABlQ5lVH1pPTde8lrImKfnZvZzmgQddXnObDCfaMbxy4y5z1tU30XEgew-2Zl2eU7E-8DNmrL3TFnY12TMPRcMoJTSzhA=='",
 "b'gAAAAABlQ5lVekQVj4SUWN6U

In [93]:
decryptMessage = [f.decrypt(elm).encode() for elm in list_of_strings]
decryptMessage

InvalidToken: 

In [82]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"A really secret message. Not for prying eyes.")
token

b'gAAAAABlQ5nFQyGFCb3oZyLneOL0cMbuBB9UeP9yK2DUeYS081zoDVH6ycsYfDwWgqrvg7P7acT1KzXG2cl3-UZqhqaWHOC1o1YVnZWjRp3VSP05GUxicx4YuNp8hQWOfhlCiw0XRYAu'

In [83]:
f.decrypt(token)

b'A really secret message. Not for prying eyes.'

In [94]:
def encryptString(key_path, string):
    with open(key_path, "rb") as key_file:
        key = key_file.read()
    f = Fernet(key)
    return f.encrypt(string.encode()).decode()

def decryptString(key_path, string):
    with open(key_path, "rb") as key_file:
        key = key_file.read()
    f = Fernet(key)
    return f.decrypt(string).decode()

tmp_string = "Hello World!"
encryptString(private_key_path, tmp_string)

'gAAAAABlQ5rIyBllsCyGQa9KwtLPUNuNtLjwLF2rBDTc4YzwNvV1gYadbFNvrjUYrxp-RwX_IIE643EE3AVCSZmNyBrpQlTfeg=='

In [95]:
decryptString(private_key_path, encryptString(private_key_path, tmp_string))

'Hello World!'

In [98]:
def encryptDataFrame(key_path, excel_path):
    df = pd.read_excel(excel_path)
    columns = df.columns
    structured_array = df.to_records(index=False)
    list_of_strings = []
    for d in structured_array:
        for elm in d:
            list_of_strings.append(str(elm))
    cryptList = [encryptString(key_path, elm) for elm in list_of_strings]
    res = []
    for i in range(0, len(cryptList), 4):
        res.append([cryptList[i], cryptList[i+1], cryptList[i+2], cryptList[i+3]])
    df = pd.DataFrame(res, columns=columns)
    crypt_xlsx = "../private/list_crypt.xlsx"
    df.to_excel(crypt_xlsx, index=False)
    return crypt_xlsx

def decryptDataFrame(key_path, excel_path):
    df = pd.read_excel(excel_path)
    columns = df.columns
    structured_array = df.to_records(index=False)
    list_of_strings = []
    for d in structured_array:
        for elm in d:
            list_of_strings.append(str(elm))
    non_crypt_list = [decryptString(key_path, elm) for elm in list_of_strings]
    res = []
    for i in range(0, len(non_crypt_list), 4):
        res.append([non_crypt_list[i], non_crypt_list[i+1], non_crypt_list[i+2], non_crypt_list[i+3]])
    df = pd.DataFrame(res, columns=columns)
    return df

excel_crypt_path = encryptDataFrame(private_key_path, "../private/login_excel.xlsx")
decryptDataFrame(private_key_path, excel_crypt_path)


,Nom,Prénom,ID,MDP
0,Harris,Tonia,tharris,123
1,Tendron,Noemie,ntendron,456
2,Harris,Erwann,eharris,789
